In [49]:
import numpy as np


def row_echelon_form(A, reduced=False):
  """
  Reduces the matrix A to row echelon form or reduced row echelon form.

  Parameters:
  A: input matrix.
  reduced: boolean, if True, returns the reduced row echelon form (RREF).
           If False, returns only the row echelon form (REF).

  Returns:
  A: matrix reduced to the desired form.
  """
  A = np.array(
      A, dtype=float)  # Convert the matrix to float to ensure precision
  rows, cols = A.shape

  # Reduce the matrix to row echelon form (REF)
  for col in range(min(rows, cols)):
    # Choose the pivot
    max_row = np.argmax(np.abs(A[col:rows, col])) + col
    if A[max_row, col] == 0:
      continue  # Skip the column if the pivot is zero

    # Swap the current row with the row containing the pivot
    A[[col, max_row]] = A[[max_row, col]]

    # Normalize the pivot row so that the pivot is 1
    A[col] = A[col] / A[col, col]

    # Eliminate the values below the pivot
    for i in range(col + 1, rows):
      A[i] -= A[i, col] * A[col]

  # If reduced is True, reduce to the reduced row echelon form (RREF)
  if reduced:
    for col in range(min(rows, cols) - 1, -1, -1):
      pivot_row = None
      for row in range(rows):
        if A[row, col] == 1:
          pivot_row = row
          break

      if pivot_row is not None:
        for row in range(rows):
          if row != pivot_row:
            A[row] -= A[row, col] * A[pivot_row]

  return A


# Testando a matriz
A = [[3, 9, 3], [1, 3, 0], [1, 3, 2]]

# Para obter a forma reduzida da matriz (RREF)
reduced_echelon_form = row_echelon_form(A, reduced=True)
print("Reduced Row Echelon Form:")
print(reduced_echelon_form)

Reduced Row Echelon Form:
[[ 1.  3.  1.]
 [ 0.  0. -1.]
 [ 0.  0.  1.]]


In [11]:
import numpy as np
import logging

# Configuração do logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')
logger = logging.getLogger()


def rref(B, tol=1e-8, debug=False):
  A = B.copy()
  rows, cols = A.shape
  r = 0
  pivots_pos = []
  row_exchanges = np.arange(rows)

  for c in range(cols):
    if debug:
      logger.debug(f"Now at row {r} and col {c} with matrix:\n{A}")

    # Find the pivot row
    pivot = np.argmax(np.abs(A[r:rows, c])) + r
    m = np.abs(A[pivot, c])
    if debug:
      logger.debug(f"Found pivot {m} in row {pivot}")

    if m <= tol:
      # Skip column c
      A[r:rows, c] = np.zeros(rows - r)
      if debug:
        logger.debug(
            f"All elements at and below ({r}, {c}) are zero.. moving on..")
    else:
      # Keep track of bound variables
      pivots_pos.append((r, c))

      if pivot != r:
        # Swap current row and pivot row
        A[[pivot, r], c:cols] = A[[r, pivot], c:cols]
        row_exchanges[[pivot, r]] = row_exchanges[[r, pivot]]

        if debug:
          logger.debug(f"Swap row {r} with row {pivot} Now:\n{A}")

      # Normalize pivot row
      A[r, c:cols] = A[r, c:cols] / A[r, c]

      # Eliminate the current column
      v = A[r, c:cols]
      # Above (before row r)
      if r > 0:
        ridx_above = np.arange(r)
        A[ridx_above, c:cols] = A[ridx_above, c:cols] - \
            np.outer(v, A[ridx_above, c]).T
        if debug:
          logger.debug(f"Elimination above performed:\n{A}")
      # Below (after row r)
      if r < rows - 1:
        ridx_below = np.arange(r + 1, rows)
        A[ridx_below, c:cols] = A[ridx_below, c:cols] - \
            np.outer(v, A[ridx_below, c]).T
        if debug:
          logger.debug(f"Elimination below performed:\n{A}")

      r += 1

    # Check if done
    if r == rows:
      break

  return A, pivots_pos, row_exchanges


M = [[0, 0, 2],
     [1, 1, 0],
     [-1, 0, 1],
     [1, 2, 0],
     [-1, -1, 2]]

# Exemplo de uso no notebook
B = np.array(M, dtype=float)

# Chamando a função com debug ativado
rref_result = rref(B, debug=True)
print("Resultado da forma reduzida por linhas:")
print(rref_result[0])
print("Posições dos pivôs:")
print(rref_result[1])
print("Trocas de linha:")
print(rref_result[2])

DEBUG: Now at row 0 and col 0 with matrix:
[[ 0.  0.  2.]
 [ 1.  1.  0.]
 [-1.  0.  1.]
 [ 1.  2.  0.]
 [-1. -1.  2.]]
DEBUG: Found pivot 1.0 in row 1
DEBUG: Swap row 0 with row 1 Now:
[[ 1.  1.  0.]
 [ 0.  0.  2.]
 [-1.  0.  1.]
 [ 1.  2.  0.]
 [-1. -1.  2.]]
DEBUG: Elimination below performed:
[[1. 1. 0.]
 [0. 0. 2.]
 [0. 1. 1.]
 [0. 1. 0.]
 [0. 0. 2.]]
DEBUG: Now at row 1 and col 1 with matrix:
[[1. 1. 0.]
 [0. 0. 2.]
 [0. 1. 1.]
 [0. 1. 0.]
 [0. 0. 2.]]
DEBUG: Found pivot 1.0 in row 2
DEBUG: Swap row 1 with row 2 Now:
[[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 2.]
 [0. 1. 0.]
 [0. 0. 2.]]
DEBUG: Elimination above performed:
[[ 1.  0. -1.]
 [ 0.  1.  1.]
 [ 0.  0.  2.]
 [ 0.  1.  0.]
 [ 0.  0.  2.]]
DEBUG: Elimination below performed:
[[ 1.  0. -1.]
 [ 0.  1.  1.]
 [ 0.  0.  2.]
 [ 0.  0. -1.]
 [ 0.  0.  2.]]
DEBUG: Now at row 2 and col 2 with matrix:
[[ 1.  0. -1.]
 [ 0.  1.  1.]
 [ 0.  0.  2.]
 [ 0.  0. -1.]
 [ 0.  0.  2.]]
DEBUG: Found pivot 2.0 in row 2
DEBUG: Elimination above performed:


Resultado da forma reduzida por linhas:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
Posições dos pivôs:
[(0, 0), (1, 1), (2, 2)]
Trocas de linha:
[1 2 0 3 4]


In [51]:
import numpy as np

# Definir vetores como colunas de uma matriz (ou linhas)
vetores = np.array([
    [4, 3, 1, 3],
    [2, 1, 4, 1],
    [5, 2, 2, 4],
    [2, 4, 3, 2]
])

# Dimensão de R^n (número de colunas da matriz)
n = vetores.shape[0]

# Calcular o posto da matriz
posto = np.linalg.matrix_rank(vetores)

# Verificar se o posto é igual a n
if posto == n:
  print("O conjunto de vetores é o span de R^n.")
else:
  print("O conjunto de vetores NÃO é o span de R^n.")

O conjunto de vetores é o span de R^n.


In [ ]:
import numpy as np

# Definir a matriz A e o vetor v
A = np.array([[]])  # Matriz 3x3
v = np.array([1, 2, 3, 4, 5])  # Vetor 3x1

# Resolver o sistema A * x = v
solucao, residuo, rank, s = np.linalg.lstsq(A, v, rcond=None)

# Verificar se há solução (resíduo pequeno ou zero)
if np.allclose(np.dot(A, solucao), v):
  print("O vetor pertence ao range da matriz.")
else:
  print("O vetor NÃO pertence ao range da matriz.")

O vetor pertence ao range da matriz.
